## Section 1: Read in files

This project requires the following modules to run.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from scipy import optimize
from scipy import stats

The import data comes form the "dat/" folder. It is imported using the following function. Two dictionaries are created, one for R200, and one for R500 data. Each data file contains simulated cluster parameters for a particular redhisft snapshot, referenced with a snapnum. The snapnum for a file is converted into a redshift value, and this is used as the dictionary key. The data in the file is subsequently read in as an ndarray, split into R200 and R500 data, and then stored in dictionaries with redshift as the key.

In [ ]:
def read_data(path):
    """Function to read in data.

    Reads in files located in the 'dat' folder. Determines redshift from the
    filename and redshifts.txt file. Stores file data in dicitonarys of
    ndarrays.

    Any data rows which contain a NaN value or a negative value are incorrect
    and therefore removed from the data.

    R200 and R500 data is stored in separate dictionarys. Follows following
    format:
        [0]: Hid
        [1]: eta
        [2]: delta
        [3]: fm

    Args:
        path (str): The folder containing the data files.

    Returns:
        R200 (dict): R200 cluster data. Keys are the red shifts of the files.
            Data is an ndarray containing parameters.
        R500 (dict): R500 cluster data. Keys are the red shifts of the files.
            Data is an ndarray containing parameters.

    """
    #: list of str: Files and directories in defined path.
    entries = os.listdir(path)
    #:  np array: list of redshift data and snapnums.
    rshifts = np.loadtxt('redshifts.txt', delimiter=' ')
    #: Correct redshift data so that negative values become 0 redshift.
    rshifts[rshifts < 0.0] = 0.0
    #: dicts: Empty dictionarys for R200 and R500 data.
    R200, R500 = {}, {}
    for entry in entries:
        print('---Reading: {}---'.format(entry), end='\r')
        #: int: Get snapnum from filename.
        snap_num = int(entry.split('_')[3])
        #: flt: Cross referenced red shift value.
        rshift = rshifts[np.where(rshifts[:,0] == snap_num)][0][2]
        entry_data = np.loadtxt(path + entry)
        entry_data = entry_data[np.logical_and(
                                               entry_data[:,1]>0.0,
                                               ~np.isnan(entry_data).any(axis=1)
                                               )]
        R200[rshift] = np.copy(entry_data[:, [0, 3, 4, 5]])
        R500[rshift] = np.copy(entry_data[:, [0, 8, 9, 10]])
    print()
    return R200, R500

R200, R500 = read_data('dat/')

## Section 2: Find the relationships between various parameters

Analysis is performed on redshift:0, R200 data. We first want to find the relationship between the virial ratio (eta), the subhalo mass fraction (fm), and the centre of mass offset (delta). We will use the Pearson coefficient to measure how correlated the parameters are. We will also save plots created to the "plots/" directory.

In [ ]:
def fit_data(x, y):
    """Method for fitting data from a file.

    Fits data linearly using the scipy optimize.curve_fit method. Also finds the
    Pearson correlation coefficient using the scipy stats.pearsonr method.
    Calculates the best fit y values by passing least squares fit parameters
    into a straight line equation.

    Args:
        x (ndarray): x values to be plotted.
        y (ndarray): y values to be plotted.

    Returns:
        y_calc (ndarray): Best fitted data points.
        popt (list): Optimal values of parameters for minimizing the sum of the
            squared residuals.
        pcov (2d array): Estimated covariance of popt.
        pcoef (flt): Pearson correlation coefficient.

    """
    #: Linear best fit line.
    def line(x, m ,c):
        return m*x+c
    #: lists of flt: Least squares fit parameters and covariances.
    popt, pcov = optimize.curve_fit(line, x, y)
    #: flt: Pearson correlation coefficient from data.
    pcoef,_ = stats.pearsonr(x,y)
    #: ndarray: Best fitted line y values.
    y_calc = line(x, popt[0], popt[1])
    return y_calc, popt, pcov, pcoef

def plot_data(x, y, y_calc, xlab, ylab, sup_title, filename):
    """Method for making plots of input data.

    Uses matplotlib to plot a scatter of raw data and then a linear straight
    line of best fit on top. The plot is then saved to the plots/ directory.

    Args:
        x (ndarray): x values.
        y (ndarray): y values.
        y_calc (ndarray): Best fit y values.
        xlab (str): Label for the x axis.
        ylab (str): Label for the y axis.
        sup_title (str): Title for the plot.
        filename (str): Name that the plot will be stored under.

    """
    #: Fig, ax objects: New figure and axis created by matplotlib.
    fig, ax = plt.subplots()
    #: Plot of points.
    ax.plot(x,y,'o',c='black',markersize=0.75)
    ax.plot(x, y_calc,c='red',alpha=0.5,linewidth=0.5)
    ax.set(
        xlabel = xlab,
        ylabel = ylab,
        title = sup_title

    )
    plt.draw()
    fig.savefig('plots/'+filename+'.png')
    
for rshift, params in R200.items():
    # Only use the zero redshift data.
    if rshift == 0.0:
        eta, delta, fm = params[:,1], params[:,2], params[:,3]
        #: fig, ax objects: New figure and axis created by matplotlib.
        fig_eta_fm, eta_fm_axs = plt.subplots(1, 2)
        # Plot linear fit.
        eta_fm_axs[0].plot(eta, fm, 'o', c='black', markersize=0.75)
        fm_calc, fm_popt, fm_cov, eta_fm_
        eta_fm_axs[1].plot(np.log(eta), np.log(fm), 'o', c='black', markersize=0.75)
        
        
        